sepration using number plate

In [1]:
import cv2
import os
from ultralytics import YOLO

def detect_license_plate(video_path):
    # Create the output directory if it doesn't exist
    if not os.path.exists('LP_frames'):
        os.makedirs('LP_frames')

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Initialize YOLO model
    model = YOLO('L_Plate.pt')
    class_name = []  # Index for numberplates


    frame_count = 0
    while True:
        # Read the next frame
        ret, frame = cap.read()

        # If the frame was not read correctly, we're done
        if not ret:
            break

        # If the frame number is even, perform object detection
        if frame_count % 1 == 0:
            # Perform object detection
            results = model(frame, conf=0.4)

            # Check if any vehicle is detected in the frame
            license_plate_detected = False
            for r in results:
                if len(r) > 0:
                    license_plate_detected = True
                    break

            # Save the frame only if a vehicle is detected
            if license_plate_detected:
                cv2.imwrite(f'LP_frames/frame_{frame_count}.jpg', frame)

        frame_count += 1

    # Release the video file
    cap.release()


detect_license_plate("120223KLM.mp4")




0: 320x640 (no detections), 238.5ms
Speed: 2.1ms preprocess, 238.5ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 203.2ms
Speed: 1.5ms preprocess, 203.2ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 208.3ms
Speed: 1.3ms preprocess, 208.3ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 211.4ms
Speed: 1.5ms preprocess, 211.4ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 214.3ms
Speed: 1.5ms preprocess, 214.3ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 199.4ms
Speed: 1.3ms preprocess, 199.4ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 313.1ms
Speed: 1.4ms preprocess, 313.1ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 331.4ms
Speed: 4.4ms prepr

In [4]:
import os
import cv2
import easyocr
import re
import shutil

def extract_register_numbers_from_folder(input_folder):
    reader = easyocr.Reader(['en'], gpu=True)  # Using English language, GPU acceleration

    # Define regular expression patterns for private and commercial vehicle registration numbers
    private_pattern = re.compile(r'[A-Z\u200c\ufeff]{2}\s?[0-9]{2}\s?[A-Z\u200c\ufeff]{1,2}\s?[0-9]{4}')
    commercial_pattern = re.compile(r'[A-Z]{3}\s?[0-9]{4}')

    # Create output folder if it doesn't exist
    output_folder = "mined_RP_numbers"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = reader.readtext(image_rgb, detail=0)

            extracted_numbers = []
            # Check for private vehicle registration number
            for text in results:
                private_match = private_pattern.search(text)
                if private_match:
                    extracted_numbers.append(private_match.group().replace(" ", ""))
                    break

            # If not a private vehicle registration number, check for commercial vehicle registration number
            if not extracted_numbers:
                for text in results:
                    commercial_match = commercial_pattern.search(text)
                    if commercial_match:
                        extracted_numbers.append(commercial_match.group().replace(" ", ""))
                        break

            if extracted_numbers:
                output_image_folder = os.path.join(output_folder, extracted_numbers[0])
                if not os.path.exists(output_image_folder):
                    os.makedirs(output_image_folder)
                shutil.copy(image_path, output_image_folder)
                print(f"Processed image {filename}, registration number: {extracted_numbers[0]}")

# Example usage
input_folder = "LP_frames"
extract_register_numbers_from_folder(input_folder)


Processed image frame_1533.jpg, registration number: KLJ4248
Processed image frame_548.jpg, registration number: ZAP5133
Processed image frame_1553.jpg, registration number: KL23U2187
Processed image frame_1552.jpg, registration number: KL23U2187
Processed image frame_1550.jpg, registration number: KL23U2187
Processed image frame_1544.jpg, registration number: KL23U2187
Processed image frame_1545.jpg, registration number: KL23U2187
Processed image frame_1551.jpg, registration number: KL23U2187
Processed image frame_1569.jpg, registration number: KL23U2187
Processed image frame_1555.jpg, registration number: KL23U2187
Processed image frame_1541.jpg, registration number: KL23U2487
Processed image frame_1540.jpg, registration number: KL23U2182
Processed image frame_1554.jpg, registration number: KL23U2187
Processed image frame_1568.jpg, registration number: KL23U2187
Processed image frame_1556.jpg, registration number: KL23U2187
Processed image frame_1557.jpg, registration number: KL23U21

In [ ]:
import cv2
import os
from ultralytics import YOLO
import easyocr
import re
import shutil

def detect_license_plate(video_path):
    if not os.path.exists('LP_frames'):
        os.makedirs('LP_frames')
    cap = cv2.VideoCapture(video_path)
    model = YOLO('L_Plate.pt')
    class_name = [] 
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % 1 == 0:
            results = model(frame, conf=0.4)
            license_plate_detected = False
            for r in results:
                if len(r) > 0:
                    license_plate_detected = True
                    break
            if license_plate_detected:
                cv2.imwrite(f'LP_frames/frame_{frame_count}.jpg', frame)
        frame_count += 1
    cap.release()



def extract_register_numbers_from_folder(input_folder):
    input_folder = "LP_frames"
    reader = easyocr.Reader(['en'], gpu=True)
    private_pattern = re.compile(r'[A-Z\u200c\ufeff]{2}\s?[0-9]{2}\s?[A-Z\u200c\ufeff]{1,2}\s?[0-9]{4}')
    commercial_pattern = re.compile(r'[A-Z]{3}\s?[0-9]{4}')
    output_folder = "mined_RP_numbers"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = reader.readtext(image_rgb, detail=0)

            extracted_numbers = []
            for text in results:
                private_match = private_pattern.search(text)
                if private_match:
                    extracted_numbers.append(private_match.group().replace(" ", ""))
                    break
            if not extracted_numbers:
                for text in results:
                    commercial_match = commercial_pattern.search(text)
                    if commercial_match:
                        extracted_numbers.append(commercial_match.group().replace(" ", ""))
                        break

            if extracted_numbers:
                output_image_folder = os.path.join(output_folder, extracted_numbers[0])
                if not os.path.exists(output_image_folder):
                    os.makedirs(output_image_folder)
                shutil.copy(image_path, output_image_folder)
                print(f"Processed image {filename}, registration number: {extracted_numbers[0]}")
